# Домашнее задание 4

## Предсказание пользовательской оценки отеля по тексту отзыва

Мы собрали для вас отзывы по 1500 отелям из совершенно разных уголков мира. Что это за отели — секрет. Вам дан текст отзыва и пользовательская оценка отеля. Ваша задача — научиться предсказывать оценку отеля по отзыву. Данные можно скачать [тут](https://www.kaggle.com/c/hseds-texts-2020/data?select=train.csv). Также я скопировал их на Google Drive (команда `gdown` приведена ниже).

Главная метрика — Mean Absolute Error (MAE). Во всех частях домашней работы (кроме нулевой) вам нужно получить значение MAE не превышающее 1. В противном случае мы будем вынуждены не засчитать задание :( 

Для измерения качества вашей модели используйте разбиение данных на train и test и замеряйте качество на тестовой части.

#### Про данные:
Каждое ревью состоит из двух текстов: `positive` и `negative` — плюсы и минусы отеля. В столбце `score` находится оценка пользователя — вещественное число 0 до 10. Вам нужно извлечь признаки из этих текстов и предсказать по ним оценку.

Удачи! 💪

**Использовать внешние данные для обучения строго запрещено. Можно использовать предобученные модели (например, из huggingface).**

In [ ]:
from pathlib import Path

import pandas as pd

In [ ]:
data_path = Path('hseds-texts-2020/train.csv')

if not data_path.exists():
    !gdown https://drive.google.com/uc?id=118Xty0jxXut7eXIScR4uE8BxWbM4kwW3
    !unzip kaggle_hseds-texts-2020.zip -d hseds-texts-2020

In [ ]:
df = pd.read_csv(data_path)
df

Предобработка текста может сказываться на качестве вашей модели.
Сделаем небольшой препроцессинг текстов: удалим знаки препинания, приведем все слова к нижнему регистру. 
Однако можно не ограничиваться этим набором преобразований. Подумайте, что еще можно сделать с текстами, чтобы помочь будущим моделям? Добавьте преобразования, которые могли бы помочь по вашему мнению.

Также мы добавили разбиение текстов на токены. Теперь каждая строка-ревью стала массивом токенов.

In [ ]:
import string

import nltk
nltk.download('punkt')

from nltk.tokenize import word_tokenize

def process_text(text):
    return [
        word
        for word in word_tokenize(text.lower())
        if word not in string.punctuation
    ]

In [ ]:
df['negative'] = df['negative'].apply(process_text)
df['positive'] = df['positive'].apply(process_text)

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df)

Пример выше делает очень простую токенизацию: разрезает текст на слова и выбрасывает пунктуацию. Есть и более умные методы. Например, метод Byte Pair Encoding (BPE) позволяет разрезать текст на токены, в которых часто встречающиеся слова будут входить целиком, а редкие окажутся разделены на несколько токенов. Эта токенизация используется, например, в небезызвестной GPT-3.

Хорошее объяснение BPE есть в [Википедии](https://en.wikipedia.org/wiki/Byte_pair_encoding), а посмотреть пример применения можно в самом начале [ноутбука](https://github.com/yandexdataschool/nlp_course/blob/28a92e376f5229fe57f6e704c9f927909265b1e2/week04_seq2seq/practice_and_homework_pytorch.ipynb) четвёртой недели курса NLP в ШАДе.

### Часть 0. Константный бейзлайн (1 балл)

Здесь мы в качестве sanity check посмотрим, какое получается MAE, если предсказывать наилучшую возможную константу.

Какая аналитическая формула для константы, минимизирущей MAE?

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred_const = <YOUR CODE>
mean_absolute_error(df_test['score'], [y_pred_const] * len(df_test))

### Часть 1. TF-IDF (1 балл)

В этой части вы для разминки можете обучить регрессию на TF-IDF векторах текстов.

**Это делать не обязательно.** TF-IDF интересен больше с исторической точки зрения, или же как простейший способ извлечь из текста хоть какие-то фичи. Если вы раньше про него не слышали, можете [прочитать](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction) в мануале scikit-learn или пропустить эту часть.

Вам понадобится как-то агрегировать фичи по положительной и отрицательной части отзыва. Если вы решите конкатенировать матрицы фичей TF-IDF, учтите, что они являются разреженными, и для их конкатенации нужно будет использовать функцию [`scipy.sparse.hstack`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.hstack.html).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDRegressor

In [ ]:
mae_regressor = SGDRegressor(
    loss='epsilon_insensitive',
    epsilon=0,
    <YOUR CODE>
)

In [ ]:
<YOUR CODE>

### Часть 2. Word2Vec (3 балла)

Обучите регрессию на усредненных Word2Vec векторах.

Примерный план (естественно, любой шаг тут можно менять):

* Скачайте какие-нибудь предобученные эмбеддинги через `gensim.downloader`. Например, `glove-twitter-100`. Полный список того, что предоставляет библиотека, смотрите [тут](https://github.com/RaRe-Technologies/gensim-data).
* Преобразуйте каждое слово из позитивной и негативной части отзыва в эмбеддинг. Усредните их внутри каждой части по отдельности.
* Сконкатенируйте эмбеддинги для позитивной и негативной части каждого отзыва.
* Обучите на том, что получится, регрессию.

In [ ]:
<YOUR CODE>

Усредняя w2v вектора, мы предполагаем, что каждое слово имеет равноценный вклад в смысл предложения, однако это может быть не совсем так. Теперь попробуйте воспользоваться другой концепцией и перевзвесить слова при получении итогового эмбеддинга текста. В качестве весов используйте IDF (inverse document frequency):

$$
\operatorname{idf}(w) = \log \frac {N} {1 + \operatorname{df}(w)},
$$

где:

* $N$ — общее количество текстов в корпусе,
* $\operatorname{df}(w)$ — количество текстов, содержащих слово $w$.

In [ ]:
def calc_idf(texts):
    <YOUR CODE>

Проведите эксперименты с размерностью эмбеддинга. Для каждого из двух методов постройте график зависимости качества модели от размерности эмбеддинга.

#### Сделайте выводы:

`<YOUR TEXT>`

**(необязательно)** Теперь попробуйте обучить регрессию на любых других эмбеддингах размерности 300 (ELMO, BERT) и сравните качество с Word2Vec.

#### Выводы:

`<YOUR TEXT>`

### Часть 3. `import torch` (6 баллов)

Теперь давайте воспользуемся более продвинутыми методами обработки текстовых данных, которые мы проходили в нашем курсе. Обучите RNN/Transformer для предсказания пользовательской оценки. Получите ошибку меньше, чем во всех вышеперечисленных методах.

Если будете обучать RNN, попробуйте ограничить максимальную длину предложения. Некоторые отзывы могут быть слишком длинными относительно остальных.

Чтобы пользоваться DataLoader, все его элементы должны быть одинаковой размерности. Для этого вы можете добавить нулевой паддинг ко всем предложениям (см пример `pad_sequence`)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
WORDS = set()
for sent in list(df['positive']):
    for w in sent:
        WORDS.add(w)
        
for sent in list(df['negative']):
    for w in sent:
        WORDS.add(w)

In [ ]:
int2word = dict(enumerate(tuple(WORDS)))
word2int = {w: i for i, w in int2word.items()}

In [ ]:
MAX_LEN = max(max(df['positive'].apply(len)), max(df['negative'].apply(len)))

In [ ]:
MAX_LEN

In [ ]:
from torch.nn.utils.rnn import pad_sequence

train_pos_pad = pad_sequence(
    [
        torch.as_tensor([word2int[w] for w in seq][:MAX_LEN])
        for seq in df_train['positive']
    ],
    batch_first=True,
)

In [ ]:
class ReviewsDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        <YOUR CODE>
        
    def __len__(self):
        <YOUR CODE>
    
    def __getitem__(self, idx):
        <YOUR CODE>

In [ ]:
BATCH_SIZE = 1

train_dataset = ReviewsDataset(df_train)
test_dataset = ReviewsDataset(df_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [ ]:
NUM_EPOCHS = 1

for n in range(NUM_EPOCHS):
    model.train()

    # Не забудьте, что в качестве метрики мы используем MAE.
    # Вам может пригодиться https://pytorch.org/docs/stable/generated/torch.nn.L1Loss.html
    
    <YOUR CODE>

### Бонус. 10 баллов

Побейте качество 0.75 в [соревновании](https://www.kaggle.com/c/hseds-texts-2020/leaderboard). Можете воспользоваться вышеперечисленными методами или попробовать что-нибудь еще.

## Acknowledgements

Ноутбук основан на [третьем домашнем задании](https://github.com/hse-ds/iad-deep-learning/blob/66fb0128da4e65cb3260c088e2d462eb9d0c5eb1/hw3/hw3.ipynb) курса ИАД DL.